In [ ]:
import random
import numpy as np

# Dados sobre a caminhada no espaço de busca
W = 0.5
c1 = 0.8
c2 = 0.9

"""
# Retire os comentário para pedir ao usuário inserir os dados de entrada
quantidade_iteracoes = int(input("Informe o número de iterações: "))
margem_erro = float(input("Informe o erro alvo: "))
quantidade_particulas = int(input("Informe o número de particulas: "))
"""

quantidade_iteracoes = 20
margem_erro = 0.005
quantidade_particulas = 20

class Particulas():
    def __init__(self):
        self.posicao = \
        np.array([(-1) ** (bool(random.getrandbits(1))) * \
                  random.random()*50, \
                  (-1)**(bool(random.getrandbits(1))) * \
                  random.random()*50])
        self.melhor_posicao_particula = self.posicao
        self.melhor_valor_particula = float('inf')
        self.velocidade = np.array([0,0])

    def __str__(self):
        print("Eu estou em ", self.posicao, " minha melhor particula é ", self.melhor_posicao_particula)

    def move(self):
        # Responsável por movimentar a partícula
        self.posicao = self.posicao + self.velocidade

class Espaco():
    def __init__(self, target, margem_erro, quantidade_particulas):
        self.target = target
        self.margem_erro = margem_erro
        self.quantidade_particulas = quantidade_particulas
        self.particulas = []
        self.melhor_valor_global = float('inf')
        self.melhor_posicao_global = np.array([random.random()*50, random.random()*50])

    def print_particulas(self):
        for particula in self.particulas:
            particula.__str__()

    def fitness(self, particula):
        return particula.posicao[0] ** 2 + particula.posicao[1] ** 2 + 1

    def set_melhor_particula(self):
        for particula in self.particulas:
            fitness_cadidate = self.fitness(particula)
            if(particula.melhor_valor_particula > fitness_cadidate):
                particula.melhor_valor_particula = fitness_cadidate
                particula.melhor_posicao_particula = particula.posicao

    def set_melhor_global(self):
        for particula in self.particulas:
            best_fitness_cadidate = self.fitness(particula)
            if(self.melhor_valor_global > best_fitness_cadidate):
                self.melhor_valor_global = best_fitness_cadidate
                self.melhor_posicao_global = particula.posicao

    def move_particulas(self):
        for particula in self.particulas:
            global W
            new_velocidade = \
            (W*particula.velocidade) + (c1*random.random()) * \
            (particula.melhor_posicao_particula - particula.posicao) + \
                            (random.random()*c2) * (self.melhor_posicao_global - particula.posicao)
            particula.velocidade = new_velocidade
            particula.move()

search_space = Espaco(1, margem_erro, quantidade_particulas)
particulas_vector = [Particulas() for _ in range(search_space.quantidade_particulas)]
search_space.particulas = particulas_vector
search_space.print_particulas()

iteracao = 0
while(iteracao < quantidade_iteracoes):
    search_space.set_melhor_particula()
    search_space.set_melhor_global()
    if(abs(search_space.melhor_valor_global - search_space.target) <= search_space.margem_erro):
        break
    search_space.move_particulas()
    iteracao += 1

print("A melhor solução é: ", search_space.melhor_posicao_global,
      " em número de iterações: ", iteracao)

In [2]:
pip install psopy

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
from psopy import minimize
from scipy.optimize import rosen
x0 = np.random.uniform(0, 2, (1000, 5))
res = minimize(rosen, x0, options={'stable_iter': 50})
res.x

array([0.99999994, 0.99999972, 0.99999946, 0.99999892, 0.99999773])

In [ ]:
# The objective function.
fun = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2
# The constraints.
cons = ({'type': 'ineq', 'fun': lambda x:  x[0] - 2 * x[1] + 2},
{'type': 'ineq', 'fun': lambda x: -x[0] - 2 * x[1] + 6},
{'type': 'ineq', 'fun': lambda x: -x[0] + 2 * x[1] + 2},
{'type': 'ineq', 'fun': lambda x: x[0]},
{'type': 'ineq', 'fun': lambda x: x[1]})
from psopy import init_feasible
x0 = init_feasible(cons, low=0., high=2., shape=(1000, 2))
res = minimize(fun, x0, constraints=cons, options={'g_rate': 1., 'l_rate': 1., 'max_velocidade': 4., 'stable_iter': 50})
res.x

array([1.39993131, 1.69996613])

In [ ]:
#https://machinelearningmastery.com/a-gentle-introduction-to-particula-swarm-optimization/

# Exemplo completo PSO com animação das partículas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
rc('animation', html='jshtml')
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation

# Função que avalia se uma partícula é boa
def f(x,y):
    return (x-3.14)**2 + (y-2.72)**2 + np.sin(3*x+1.41) + np.sin(4*y-1.73)

# Faz o cálculo do gráfico em 3D com [0,5]X[0,5]
x, y = np.array(np.meshgrid(np.linspace(0,5,100), np.linspace(0,5,100)))
z = f(x, y)

# Encontra o menor valor entre todas as partículas
# A função ravel transforma uma matriz em um vetor (resumidamente)
x_min = x.ravel()[z.argmin()]
y_min = y.ravel()[z.argmin()]

# Parâmetros do espaço de busca do algoritmos
c1 = c2 = 0.1
w = 0.8

# Cria as partículas
quantidade_particulas = 20
"""
A função seed (semente), é uma função que garante que toda inicialização de
valores aleatórios seja o mesmo. Isso garante que conseguiremos testar as mesmas
velocidades e posições para diferentes tipos de parâmetros. Isso é usado para
permitir realizar a comparação de testes.
"""
np.random.seed(100)

# Define a posição e velocidade inicial de maneira aleatória
X = np.random.rand(2, quantidade_particulas) * 5
V = np.random.randn(2, quantidade_particulas) * 0.1

# Inicializa os dados
melhor_particula = X
melhor_particula_obj = f(X[0], X[1])
melhor_global = melhor_particula[:, melhor_particula_obj.argmin()]
melhor_global_obj = melhor_particula_obj.min()

def update():
    "Function to do one iteracao of particula swarm optimization"
    global V, X, melhor_particula, melhor_particula_obj, melhor_global, melhor_global_obj
    # Update params
    r1, r2 = np.random.rand(2)
    V = w * V + c1*r1*(melhor_particula - X) + c2*r2*(melhor_global.reshape(-1,1)-X)
    X = X + V
    obj = f(X[0], X[1])
    melhor_particula[:, (melhor_particula_obj >= obj)] = X[:, (melhor_particula_obj >= obj)]
    melhor_particula_obj = np.array([melhor_particula_obj, obj]).min(axis=0)
    melhor_global = melhor_particula[:, melhor_particula_obj.argmin()]
    melhor_global_obj = melhor_particula_obj.min()

# Configura o gráfico: Mapa de contorno
fig, ax = plt.subplots(figsize=(8,6))
fig.set_tight_layout(True)
img = ax.imshow(z, extent=[0, 5, 0, 5], origin='lower', cmap='viridis', alpha=0.5)
fig.colorbar(img, ax=ax)
ax.plot([x_min], [y_min], marker='x', markersize=5, color="white")
contours = ax.contour(x, y, z, 10, colors='black', alpha=0.4)
ax.clabel(contours, inline=True, fontsize=8, fmt="%.0f")

melhor_particula_plot = ax.scatter(melhor_particula[0], melhor_particula[1], marker='o', color='black', alpha=0.5)
p_plot = ax.scatter(X[0], X[1], marker='o', color='blue', alpha=0.5)
p_arrow = ax.quiver(X[0], X[1], V[0], V[1], color='blue', width=0.005, angles='xy', scale_units='xy', scale=1)
melhor_global_plot = plt.scatter([melhor_global[0]], [melhor_global[1]], marker='*', s=100, color='black', alpha=0.4)

ax.set_xlim([0,5])
ax.set_ylim([0,5])

# Define a animação
def animate(i):
    "Passos do PSO: Atualização do algoritmo e exibindo no gráfico"
    title = 'Iteração {:02d}'.format(i)
    # Atualizar os parâmetros
    update()
    # Define a figura
    ax.set_title(title)
    melhor_particula_plot.set_offsets(melhor_particula.T)
    p_plot.set_offsets(X.T)
    p_arrow.set_offsets(X.T)
    p_arrow.set_UVC(V[0], V[1])
    melhor_global_plot.set_offsets(melhor_global.reshape(1,-1))
    return ax, melhor_particula_plot, p_plot, p_arrow, melhor_global_plot
# Demora alguns segundo para gerar a animação, após isso, pressione no botão de executar
anim = animation.FuncAnimation(fig, animate, frames=list(range(1,50)), interval=500, blit=False, repeat=True)
anim

In [ ]:
print("A melhor solução do PSO foi f({})={}".format(melhor_global, melhor_global_obj))
print("O melhor global foi f({})={}".format([x_min,y_min], f(x_min,y_min)))